In [1]:
import os
from silence_tensorflow import silence_tensorflow
import numpy as np
from transformers import TFAutoModel
from sklearn import metrics
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from transformers import TFAutoModel, AutoTokenizer
from tqdm.notebook import tqdm
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dropout, Conv1D, Embedding, SpatialDropout1D, concatenate, MaxPooling1D, Conv1D
from tensorflow.keras.layers import GRU, LSTM,Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
from tensorflow.keras.layers import LSTM, GRU, Dropout, Flatten
from tensorflow.keras import backend as K
from sklearn.model_selection import KFold
from tensorflow.keras.callbacks import Callback
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Lambda
import warnings
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tqdm.notebook import tqdm
import gc
import wandb
from wandb.keras import WandbCallback
np.random.seed(21)
warnings.filterwarnings('ignore')
silence_tensorflow()

In [ ]:
'''Check if directory exists, if not, create it'''
import os
MYDIR = ("/scratch/arjunth2001")
CHECK_FOLDER = os.path.isdir(MYDIR)
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")

In [2]:
BATCH_SIZE = 4
model_name = 'vinai/bertweet-base'
tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=True)
MAX_LEN = 125

emoji is not installed, thus not converting emoticons or emojis into text. Please install emoji: pip3 install emoji
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
train =pd.read_csv("../data/task1b/train.csv")
FEATURES=["dale_chall","bad_words", "num_words","all_caps",	"emoji"	,"capitals","total_length","caps_vs_length","num_unique_words",	"words_vs_unique"	,"num_urls",	"!",	"?"]
features = train[FEATURES].fillna(0)
ss = StandardScaler()
ss.fit(features)
features = ss.transform(features)

In [4]:
#Function for cutting off the middle part of long texts.
def text_process(text):
    ws = text.split(' ')
    if(len(ws)>130):
        text = ' '.join(ws[:90]) + ' ' + ' '.join(ws[-40:])
    return text

In [5]:
y_train = train[["NONE","OFFN","HATE","PRFN"]].values
X_train = train['c_text'].apply(lambda x: text_process(str(x))).fillna("something").values.tolist()

In [6]:
def tokenize_sentences(sentences, tokenizer, max_seq_len = 125 ):
    tokenized_sentences = []

    for sentence in tqdm(sentences):
        tokenized_sentence = tokenizer.encode(
                            sentence,
                            truncation=True,               
                            add_special_tokens = True, 
                            max_length = max_seq_len,
                    )
        
        tokenized_sentences.append(tokenized_sentence)

    return np.array(tokenized_sentences)

def create_attention_masks(tokenized_and_padded_sentences):
    attention_masks = []

    for sentence in tokenized_and_padded_sentences:
        att_mask = [int(token_id > 0) for token_id in sentence]
        attention_masks.append(att_mask)

    return np.asarray(attention_masks)
def regular_encode(texts,tokenizer,maxlen=MAX_LEN):
  input_ids = tokenize_sentences(texts, tokenizer, MAX_LEN)
  input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
  attention_masks = create_attention_masks(input_ids)
  return input_ids,attention_masks

In [7]:
x_train,x_train_att = regular_encode(X_train,tokenizer,maxlen=MAX_LEN)
i=0

  0%|          | 0/768 [00:00<?, ?it/s]

  0%|          | 0/3075 [00:00<?, ?it/s]

In [8]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()
        self.interval = interval
        self.X_val, self.y_val = validation_data
        self.max_score = 0
        self.not_better_count = 0

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=1)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: %d - score: %.6f \n" % (epoch+1, score))
            if (score > self.max_score):
                print("*** New High Score (previous: %.6f) \n" % self.max_score)
                model.save_weights(f"/scratch/arjunth2001/ft_fine_1b_cnn_{i}.h5")
                self.max_score=score
                self.not_better_count = 0
            else:
                self.not_better_count += 1
                if self.not_better_count > 3:
                    print("Epoch %05d: early stopping, high score = %.6f" % (epoch,self.max_score))
                    self.model.stop_training = True

In [9]:
def get_model(bert_model, features ,clipvalue=1.,num_filters=40,dropout=0.5,max_len=125):
    import tensorflow as tf
    features_input = Input(shape=(features.shape[1],))
    input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    attention_masks = Input(shape=(max_len,), dtype=tf.int32, name="input_att_masks")
    bert_output = bert_model(input_ids, attention_mask=attention_masks)
    hidden_states= bert_output.hidden_states
    last_four_layers = [hidden_states[i] for i in (-1, -2, -3, -4)]
    x= tf.concat(last_four_layers,-1)
    convs = []
    filter_sizes = [2,3,4,5]
    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=128, kernel_size=filter_size, activation='relu')(x)
        l_pool = MaxPooling1D(pool_size=3)(l_conv)
        convs.append(l_pool)
    l_merge = concatenate(convs,axis=1)
    x = Dropout(0.5)(l_merge)  
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = concatenate([x ,features_input])
    outp = Dense(4, activation="softmax")(x)
    model = Model(inputs=[input_ids,attention_masks,features_input], outputs=outp)
    import tensorflow as tf
    adam = tf.optimizers.Adam(clipvalue=clipvalue)
    model.compile(loss='binary_crossentropy',
                  optimizer=adam,
                  metrics=['accuracy'])
    return model

In [10]:
transformer_layer = TFAutoModel.from_pretrained(model_name,output_hidden_states=True)
transformer_layer.compile()
model = get_model(transformer_layer, features)
model.summary()
epochs = 100
import json
with open("../data/splits_b.json") as f:
    splits = json.load(f)    
gc.collect()
K.clear_session()
x_train=np.asarray(x_train).astype(np.float32)
for train_index, test_index in splits:
    train_index = np.array(train_index)
    test_index = np.array(test_index)
    i+=1
    print(f"fold: {i}")
    kfold_y_train,kfold_y_test = y_train[train_index], y_train[test_index]
    kfold_X_train = x_train[train_index]
    kfold_X_train_att = x_train_att[train_index]
    kfold_X_features = features[train_index]
    kfold_X_valid = x_train[test_index]
    kfold_X_valid_att = x_train_att[test_index]
    kfold_X_valid_features = features[test_index] 
    gc.collect()
    K.clear_session()
    del model
    model = get_model(transformer_layer, features)
    ra_val = RocAucEvaluation(validation_data=([kfold_X_valid,kfold_X_valid_att,kfold_X_valid_features], kfold_y_test), interval = 1)
    model.fit([kfold_X_train,kfold_X_train_att,kfold_X_features], kfold_y_train, batch_size=BATCH_SIZE, epochs=epochs, verbose=1,
            callbacks = [ra_val])
    gc.collect()
print("Done")

Some layers from the model checkpoint at vinai/bertweet-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at vinai/bertweet-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 125)]        0                                            
__________________________________________________________________________________________________
input_att_masks (InputLayer)    [(None, 125)]        0                                            
__________________________________________________________________________________________________
tf_roberta_model (TFRobertaMode TFBaseModelOutputWit 134899968   input_word_ids[0][0]             
                                                                 input_att_masks[0][0]            
__________________________________________________________________________________________________
tf_op_layer_concat (TensorFlowO [(None, 125, 3072)]  0           tf_roberta_model[0][13

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: arjunth2001 (use `wandb login --relogin` to force relogin)


49/49 [==============================] - 9s 187ms/step

 ROC-AUC - epoch: 1 - score: 0.654408 

*** New High Score (previous: 0.000000) 

192/192 [==============================] - 5s 24ms/step


"val score for fold 1, epoch: 1",0.65441
_runtime,104
_timestamp,1629094996
_step,1
epoch,0
loss,0.57927
accuracy,0.34808


...

fold: 2


49/49 [==============================] - 7s 150ms/step

 ROC-AUC - epoch: 1 - score: 0.649298 

*** New High Score (previous: 0.000000) 

192/192 [==============================] - 5s 24ms/step


"val score for fold 2, epoch: 1",0.6493
_runtime,86
_timestamp,1629095095
_step,1
epoch,0
loss,0.56907
accuracy,0.38296


"val score for fold 2, epoch: 1",▁
_runtime,▁█
_timestamp,▁█
_step,▁█
epoch,▁
loss,▁
accuracy,▁


Done
